<a href="https://colab.research.google.com/github/mahmoudshaddad/Master-/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Standard library import for interacting with the operating system.
import os

# Import PyTorch for deep learning models and tensor computations.
import torch

# Load and manipulate datasets conveniently with Hugging Face's dataset library.
from datasets import load_dataset

# AutoModelForCausalLM for automatic causal language model loading based on a given model name.
# AutoTokenizer for automatic tokenizer loading matching the model's expected input format.
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    # BitsAndBytesConfig to configure the bitsandbytes library for efficient training.
    BitsAndBytesConfig,
    # HfArgumentParser for parsing command-line arguments to Hugging Face model training scripts.
    HfArgumentParser,
    # TrainingArguments to specify and store hyperparameters for model training.
    TrainingArguments,
    # Pipeline for easy inference with models.
    pipeline,
    # Logging for configuring the logging level of the transformers library.
    logging,
)

# LoraConfig for configuring LoRA (Low-Rank Adaptation) parameters within PEFT (Prompt Engineering and Fine-Tuning).
from peft import LoraConfig, PeftModel

# SFTTrainer for Sparse Fine-Tuning of models, enabling efficient training on small datasets.
from trl import SFTTrainer

from datasets import load_dataset
from huggingface_hub import notebook_login


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [ ]:
# here we define the model that we use to finetuning job which is LLaMa Chat
model_name = "NousResearch/Llama-2-7b-chat-hf"
fineModel_name = "MahmoudShaddad/DIN-Llama-2-7b-chat-finetunewith2epoch"
dataset_name = "MahmoudShaddad/llama2QA"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset, DatasetDict



# Load the 'train' split of the dataset
dataset = load_dataset(dataset_name, split='train')

# Split the dataset into training and test sets
split_data = dataset.train_test_split(test_size=0.01)  # the test size usually is 0.2

# Create a DatasetDict to organize the splits
split_dataset = DatasetDict({
    'train': split_data['train'],
    'test': split_data['test']
})

# Access the train and test datasets
train_ds = split_dataset['train']
test_ds = split_dataset['test']

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
test_ds

Dataset({
    features: ['text'],
    num_rows: 13
})

# For the Base Model

In [ ]:
# pipe = pipeline(task="text-generation", model=model_name, tokenizer=tokenizer, max_length=200)


In [ ]:
import re
def extract_reference_answer(text):
    # Adjusted pattern to match any characters until the "</s>" marker
    pattern = r"\[/INST\](.*?)\s*</s>"
    match = re.search(pattern, text, re.DOTALL)  # DOTALL to match across lines
    if match:
        return match.group(1).strip()  # Removes leading/trailing whitespace
    else:
        return None

# Apply the corrected function to extract reference answers
reference_answers_test = [extract_reference_answer(example['text']) for example in test_ds]
reference_answers_train = [extract_reference_answer(example['text']) for example in dataset]

reference_answers_test = [ans for ans in reference_answers_test if ans is not None]
reference_answers_train = [ans for ans in reference_answers_train if ans is not None]


# Check the length or a few elements to verify
# reference_answers_test
len(reference_answers_test)


13

In [ ]:
reference_answers_test

['Die DIN EN 206 normiert die Festigkeitsklassen, die Dichte, die Druckfestigkeit, die Wasserdurchlässigkeit sowie die Frost- und Frosttausalzbeständigkeit von Beton.',
 'Die DIN 1045-2 definiert die Expositionsklassen XC, XD, XS und XF für die unterschiedlichen Umwelteinflüsse, denen Betonbauteile ausgesetzt sein können.',
 'Die DIN 1045-2 legt Anforderungen an Tragfähigkeit, Gebrauchstauglichkeit, Dauerhaftigkeit und Feuerwiderstand von Betonkonstruktionen fest. Sie behandelt außerdem Materialanforderungen, Grundlagen der Tragwerksplanung, Bemessung und Konstruktion von Bauteilen sowie Ausführung und Instandsetzung von Stahlbeton- und Spannbetonbauwerken.',
 'Die DIN EN 1992-1-2 legt Anforderungen an die Tragfähigkeit, Gebrauchstauglichkeit und Dauerhaftigkeit von Stahlbeton- und Spannbetonbauwerken fest.',
 'In der DIN EN 1992-1-2 werden Anforderungen an den Brandschutz von Betonbauteilen, insbesondere hinsichtlich Feuerwiderstandsdauer und Schutzmaßnahmen, festgelegt.',
 'Die Festi

In [ ]:
import re
def extract_question(text):
    # Pattern to match the text between [INST] and [/INST]
    pattern = r"\[INST\](.*?)\[/INST\]"

    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)

    # If a match is found, return the first group (the matched text without the tags)
    if match:
        return match.group(1).strip()  # .strip() removes leading/trailing whitespaces
    else:
        return None  # Or return an empty string or any indication that no match was found
# Extract questions for each row in the dataset and store them in a list
questions_test= [extract_question(example['text']) for example in test_ds]
questions_train= [extract_question(example['text']) for example in dataset]


# Filter out None values if there are entries without [INST] tags
questions_test = [q for q in questions_test if q is not None]
questions_train = [q for q in questions_train if q is not None]



len(questions_test)

13

In [ ]:
questions_test

['Welche Eigenschaften von Beton werden in der DIN EN 206 normiert?',
 'Frage: Welche Klassen von Expositionsklassen werden in der DIN 1045-2 für Betonbauteile definiert?',
 'Text: DIN 1045-2 ist die deutsche Baunorm, die die Bemessung und Konstruktion von Stahlbeton- und Spannbetontragwerken regelt. Die Norm legt Anforderungen an die Tragfähigkeit, Gebrauchstauglichkeit, Dauerhaftigkeit und Feuerwiderstand von Betonkonstruktionen fest. Sie behandelt Aspekte wie Materialanforderungen, Grundlagen der Tragwerksplanung, Bemessung und Konstruktion von Bauteilen sowie Ausführung und Instandsetzung von Stahlbeton- und Spannbetonbauwerken.\n\nFrage: Welche Aspekte umfasst die DIN 1045-2 im Bereich der Betonkonstruktionen?',
 'Frage: Was regelt die DIN EN 1992-1-2 in Bezug auf den Stahlbeton- und Spannbetonbau?',
 'Welche Anforderungen werden in der DIN EN 1992-1-2 bezüglich des Brandschutzes von Betonbauteilen festgelegt?',
 'Text: \nDas europäische Regelwerk DIN EN 206 legt Anforderungen an 

In [ ]:


# def generate_model_answers(questions):
#     generated_answers = []
#     for idx, question in enumerate(questions):
#         print(f"Processing question number: {idx + 1}")
#         result = pipe(f"<s>[INST] {question} [/INST]")
#         generated_answer = result[0]['generated_text']
#         # print(generated_answer)
#         generated_answers.append(generated_answer)
#     return generated_answers
# # Generate answers for the test questions
# generated_answers_testBaseModel = generate_model_answers(questions_test)


In [ ]:

# import json



# # # # Specify the filename
# filename = 'modelResponetestBaseModel.json'

# # # # Open a file in write mode ('w') and use json.dump() to write the list to the file
# with open(filename, 'w', encoding='utf-8') as f:
#   json.dump(generated_answers_testBaseModel, f, ensure_ascii=False, indent=4)


# For the fine tuning Model

In [ ]:
pipe = pipeline(task="text-generation", model=fineModel_name, tokenizer=tokenizer, max_length=200)


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:

def generate_model_answers(questions):
    generated_answers = []
    for idx, question in enumerate(questions):
        print(f"Processing question number: {idx + 1}")
        result = pipe(f"<s>[INST] {question} [/INST]")
        generated_answer = result[0]['generated_text']
        # print(generated_answer)
        generated_answers.append(generated_answer)
    return generated_answers
# Generate answers for the test questions
generated_answers_testfinetunedmodel = generate_model_answers(questions_test)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Processing question number: 1
Processing question number: 2
Processing question number: 3
Processing question number: 4
Processing question number: 5
Processing question number: 6
Processing question number: 7
Processing question number: 8
Processing question number: 9
Processing question number: 10
Processing question number: 11
Processing question number: 12
Processing question number: 13


In [ ]:

import json



# # # Specify the filename
filename = 'modelResponetestfinetunedModel.json'

# # # Open a file in write mode ('w') and use json.dump() to write the list to the file
with open(filename, 'w', encoding='utf-8') as f:
  json.dump(generated_answers_testfinetunedmodel, f, ensure_ascii=False, indent=4)
